In [1]:
import json
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/prelabeling-using-gpt/labeling_df_part1.csv
/kaggle/input/prelabeling-using-gpt/labeling_df_part5.csv
/kaggle/input/prelabeling-using-gpt/labeling_df_part3.csv
/kaggle/input/prelabeling-using-gpt/labeling_df_part2.csv
/kaggle/input/prelabeling-using-gpt/labeling_df_part4.csv
/kaggle/input/batch-output-converting/gpt4_part1_output_2k.jsonl
/kaggle/input/batch-output-converting/gpt4_part1_output_10k-12k.jsonl.jsonl
/kaggle/input/batch-output-converting/gpt4_part1_output_14k-16k.jsonl.jsonl
/kaggle/input/batch-output-converting/part1_outputt.jsonl
/kaggle/input/batch-output-converting/gpt4_part1_output_6k-8k.jsonl.jsonl
/kaggle/input/batch-output-converting/gpt4_part1_output_4k-6k.jsonl.jsonl
/kaggle/input/batch-output-converting/gpt4_part1_output_16k-18k.jsonl.jsonl
/kaggle/input/batch-output-converting/gpt4_part1_output_18k-20k.jsonl.jsonl
/kaggle/input/batch-output-converting/gpt4_part1_output_8k-10k.jsonl.jsonl
/kaggle/input/batch-output-converting/labeled_df_25k.csv
/ka

In [114]:
selected_df=pd.read_csv("/kaggle/input/prelabeling-using-gpt/labeling_df_part1.csv")
df = selected_df.iloc[18000:20000].copy().reset_index()

In [63]:
#df['content'] = df['content'].str.replace('<mentionstart>', '').str.replace('<mentionmiddle>', '').str.replace('<mentionend>', '')

In [116]:
# Load the JSONL file
with open('/kaggle/input/batch-output-converting/gpt4_part1_output_18k-20k.jsonl.jsonl', 'r') as file:
    json_lines = file.readlines()

In [117]:
# Parse each JSON object
responses = [json.loads(line) for line in json_lines]

data = []

# Assuming 'responses' is a list containing response objects
for response in responses:
    custom_id = response['custom_id']
    choices = response['response']['body']['choices'][0]['message']['content'].split('\n')
    
    for i, choice in enumerate(choices):
        tweet_number = (int(custom_id.split('-')[1]) - 1) * 250 + (i)  # Calculate the tweet number
        data.append({
            'tweet_number': tweet_number,
            **json.loads(choice.split('. ')[1])
        })
        
# Create a DataFrame from the extracted data
classified_df = pd.DataFrame(data)

# Merge the original DataFrame with the classified DataFrame
classified_df['tweet_number'] = classified_df['tweet_number']
final_df = df.merge(classified_df, left_index=True, right_on='tweet_number')
final_df.set_index('index', inplace=True)

# Drop the tweet_number column as it is no longer needed
final_df.drop(columns=['tweet_number','level_0'], inplace=True)
final_df.to_csv('part1_labeled_18k-20k.csv', index=True)

# Display the final DataFrame
print(final_df.head())
print(final_df.tail())

                                                 content  S    C  T
index                                                              
18000  Yesus membuat yang ga ada menjadi ada, Prabowo...  1  [2]  6
18001  <mentionstart> Rakyat tahu, MASBowoGBRAN Palin...  2  [2]  6
18002  <mentionstart> Ini yg jd capres Prabowo atau J...  5  [0]  0
18003  Ngawi : 18012024. Warga spontan berebut menyal...  2  [3]  8
18004  <mentionstart> pak prabowo menginspirasi banya...  2  [2]  6
                                                 content  S    C  T
index                                                              
19995  Usai TKN Prabowo-Gibran Luncurkan Ratusan Beca...  5  [2]  0
19996  Gibran Rakabuming adalah sosok pemimpin yang t...  2  [2]  6
19997  <mentionstart> <mentionmiddle> <mentionmiddle>...  1  [2]  0
19998  Kalau mau jadi Capres, ya harus punya rekam je...  1  [2]  6
19999  <mentionstart> Prabowo, seperti panji-panji ya...  2  [2]  6
